In [ ]:
#import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt

import numpy as np
import sys
sys.path.insert(0,'/local2/home/tong/caffe-master/python')
import caffe

import os
from matplotlib import rc
rc('mathtext', default='regular')

from sklearn.metrics import confusion_matrix

caffe.set_mode_gpu()
caffe.set_device(0)


In [ ]:
#for training
solver = caffe.SGDSolver('/local2/home/tong/fashionRecommendation/models/fashionNet_3/fashion_solver_3.prototxt')
#CHANGE!!!
# solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
#CHANGE!!!
# solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
# solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/fashion_params_2_91448.caffemodel')
# solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/fashion_params_2_91448.caffemodel')


In [ ]:
def test_avg(test_iter,iters):
    avg_accu = 0
    avg_loss = 0
    y_l_true = []
    y_l_pred = []
    y_r_true = []
    y_r_pred = []
    for i in range(0,test_iter):
        if (i%50==0 or (i==(test_iter-1))):
            print("{}/{} @ {}\n".format(i+1,test_iter,iters))
        solver.test_nets[0].forward()
        avg_accu = avg_accu + solver.test_nets[0].blobs['accuracy'].data
        avg_loss = avg_loss + solver.test_nets[0].blobs['rank_Loss'].data
        for j in range(0,len(solver.test_nets[0].blobs['label_top'].data)):
            y_l_pred.append(0 if solver.test_nets[0].blobs['metric_fc3'].data[j][0]>solver.test_nets[0].blobs['metric_fc3'].data[j][1] else 1)
            y_l_true.append(1)
            y_r_pred.append(0 if solver.test_nets[0].blobs['metric_fc3_n'].data[j][0]>solver.test_nets[0].blobs['metric_fc3_n'].data[j][1] else 1)
            y_r_true.append(0)
    cMat_l = confusion_matrix(y_l_true, y_l_pred)
    if(len(cMat_l)==1):
        cMat_l = [[0,0],[0,cMat_l[0][0]]]
    cMat_r = confusion_matrix(y_r_true, y_r_pred)
    if(len(cMat_r)==1):
        cMat_r = [[cMat_r[0][0],0],[0,0]]
    return (avg_accu/test_iter), (avg_loss/test_iter),cMat_l,cMat_r


In [ ]:
# test_iter = 450
# test_accu,test_loss,cMat_l,cMat_r=test_avg(test_iter)
# print("test_accu: {}\n".format(test_accu))
# print("test_loss: {}\n".format(test_loss))
# print("cMat_l:\n")
# print(cMat_l)
# print("\ncMat_r:\n")
# print(cMat_r)


In [ ]:
recordDir = '/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/'
test_interval = 184 # 1/50 train epoch
test_iter = 450 # 1 test epoch
start_iter = 0
end_iter = start_iter+9178*(7+3) #7+3 epoch
for i in range(start_iter,end_iter+1):
    if (i%test_interval==0 or i==end_iter):
        #validation & conf_matrix
        solver.test_nets[0].copy_from(recordDir+'fashion_params_2_'+str(i)+'.caffemodel')
        test_accu,test_loss,cMat_l,cMat_r=test_avg(test_iter,i)
        test_rank_accu = open(recordDir + 'test_rank_accu.txt','a')
        test_rank_loss = open(recordDir + 'test_rank_loss.txt','a')
        test_rank_accu.write(str(i)+' '+str(test_accu)+'\r\n')
        test_rank_loss.write(str(i)+' '+str(test_loss)+'\r\n')
        test_rank_accu.close()
        test_rank_loss.close()
        te_l_conf = open(recordDir + 'conf_l_matrix.txt','a')
        te_l_conf.write(str(i)+' '+str(cMat_l[0][0])+' '+str(cMat_l[0][1])+' '+str(cMat_l[1][0])+' '+str(cMat_l[1][1])+'\r\n')
        te_l_conf.close()
        te_r_conf = open(recordDir + 'conf_r_matrix.txt','a')
        te_r_conf.write(str(i)+' '+str(cMat_r[0][0])+' '+str(cMat_r[0][1])+' '+str(cMat_r[1][0])+' '+str(cMat_r[1][1])+'\r\n')
        te_r_conf.close()   
        print("\nIters done:{}, VAL_accu={}, VAL_loss={}.\n".format(i,test_accu,test_loss))
        #visualization
        test_accu_whole = []
        test_loss_whole = []
        test_accu_whole = open(recordDir+'test_rank_accu.txt').readlines()
        test_loss_whole = open(recordDir+'test_rank_loss.txt').readlines()
        test_iter_idx = []
        test_loss = []
        test_accuracy = []
        for i in range(0, len(test_accu_whole)):
            #test_iter_idx
            test_iter_idx.append(int(test_accu_whole[i].strip('\r\n').split(' ')[0]))
            #test_loss
            test_loss.append(float(test_loss_whole[i].strip('\r\n').split(' ')[1]))
            #test_accuracy
            test_accuracy.append(float(test_accu_whole[i].strip('\r\n').split(' ')[1]))
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(test_iter_idx, test_loss, '--gp', label = 'Avg_V_Loss')
        ax_right = ax_left.twinx()
        ax_right.plot(test_iter_idx, test_accuracy, '-yp', label = 'Avg_V_Accuracy')
        # ask matplotlib for the plotted objects and their labels
        lines_left, labels_left = ax_left.get_legend_handles_labels()
        lines_right, labels_right = ax_right.get_legend_handles_labels()
        ax_right.legend(lines_left + lines_right, labels_left + labels_right, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Loss")
        ax_right.set_ylabel("Accuracy")
        ax_right.set_title("HVA@({:.3f},{}), LVL@({:.3f},{})".format(max(test_accuracy),test_iter_idx[test_accuracy.index(max(test_accuracy))],min(test_loss),test_iter_idx[test_loss.index(min(test_loss))]))
        plt.savefig(recordDir+'rank_record.png', bbox_inches='tight')   


In [ ]:
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
import numpy as np

recordDir = '/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/'

conf_l_matrix_whole = []
conf_r_matrix_whole = []
conf_l_matrix_whole = open(recordDir+'conf_l_matrix.txt').readlines()
conf_r_matrix_whole = open(recordDir+'conf_r_matrix.txt').readlines()

conf_matrix = []
for i in range(0,len(conf_l_matrix_whole)):
    conf_matrix.append([int(conf_r_matrix_whole[i].split(' ')[1]),int(conf_r_matrix_whole[i].split(' ')[2]), \
                        int(conf_l_matrix_whole[i].split(' ')[3]),int(conf_l_matrix_whole[i].split(' ')[4])])

fig,ax = plt.subplots()
im = ax.imshow(np.array(conf_matrix).T, interpolation='nearest', cmap=plt.cm.jet)
fig.colorbar(im, ax=ax)
plt.xlabel('Training Iterations Done(n)')
plt.ylabel('like-1(like,dislike), dislike-0(like,dislike)')
plt.title('Confusion Matrix@(dislike,like)')
plt.savefig(recordDir+'cMat.png',bbox_inches='tight')
